In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
data= pd.read_csv(r"C:\Users\HP\Downloads\HS_VaR.csv")
df=pd.DataFrame(data=data)
df.tail()

,Day,1,2,3
248,249,0.0637,0.0692,0.0706
249,250,0.0660,0.0677,0.0708
250,251,0.0656,0.0728,0.0732
251,252,0.0631,0.0746,0.0719
252,Today,0.0620,0.0660,0.0700


In [3]:
df= df.dropna()
df.head()

,Day,1,2,3
0,1,0.0600,0.0650,0.0700
1,2,0.0649,0.0747,0.0737
2,3,0.0693,0.0698,0.0701
3,4,0.0610,0.0712,0.0756
4,5,0.0600,0.0724,0.0766


In [4]:
df_EoD = pd.DataFrame(df.tail(1))
df_EoD

,Day,1,2,3
252,Today,0.062,0.066,0.07


In [5]:
#bond parameters
FV = 100
T = 3
coupon = 5
Freq = "Annual"

In [6]:
#Bond pricing today
pv_coupons = coupon*(math.exp(-1*df_EoD.iloc[0,1])+math.exp(-2*df_EoD.iloc[0,2])+math.exp(-3*df_EoD.iloc[0,3]))
pv_facevalue = FV* math.exp(-3* df_EoD.iloc[0,3])
mtm_today = pv_coupons+pv_facevalue

In [7]:
mtm_today

94.19246523622196

In [8]:
#simulation logic
df= df[:-1]
df.head()

,Day,1,2,3
0,1,0.0600,0.0650,0.0700
1,2,0.0649,0.0747,0.0737
2,3,0.0693,0.0698,0.0701
3,4,0.0610,0.0712,0.0756
4,5,0.0600,0.0724,0.0766


In [9]:
df= df.drop(df.columns[0],axis=1)
df.head()

,1,2,3
0,0.0600,0.0650,0.0700
1,0.0649,0.0747,0.0737
2,0.0693,0.0698,0.0701
3,0.0610,0.0712,0.0756
4,0.0600,0.0724,0.0766


In [10]:
df = df.pct_change(1)
df.head()

,1,2,3
0,NaN,NaN,NaN
1,0.081667,0.149231,0.052857
2,0.067797,-0.065596,-0.048847
3,-0.119769,0.020057,0.078459
4,-0.016393,0.016854,0.013228


In [11]:
df = df + 1
df.head()

,1,2,3
0,NaN,NaN,NaN
1,1.081667,1.149231,1.052857
2,1.067797,0.934404,0.951153
3,0.880231,1.020057,1.078459
4,0.983607,1.016854,1.013228


In [12]:
df.tail()

,1,2,3
247,1.048137,1.030942,0.987968
248,0.943704,0.944065,0.955345
249,1.036107,0.978324,1.002833
250,0.993939,1.075332,1.033898
251,0.961890,1.024725,0.982240


In [13]:
#simulatedd risk facotrs generation
df_EoD = df_EoD.drop(df_EoD.columns[0],axis=1)
df_EoD

,1,2,3
252,0.062,0.066,0.07


In [14]:
df["1"] = df["1"] * df_EoD.iloc[0,0]
df["2"] = df["2"] * df_EoD.iloc[0,1]
df["3"] = df["3"] * df_EoD.iloc[0,2]

In [15]:
df["Sim_PL"]=0
df["Sim_MTM"]=0

In [16]:
#sim mtm generation
for row in range(0,len(df)):
    R1= df.iloc[row]["1"]
    R2= df.iloc[row]["2"]
    R3= df.iloc[row]["3"]

    sim_coupons = coupon * (math.exp(-1*R1) + math.exp(-2*R2) + math.exp(-3*R3))
    sim_facevalue = FV * math.exp(-3*R3)
    sim_mtm = sim_coupons + sim_facevalue
    
    df.iloc[row,df.columns.get_loc("Sim_MTM")] = sim_mtm
    
df["Sim_PL"] = mtm_today - df["Sim_MTM"]

#sorting the sim pl numbers in ascending order

df =df.sort_values(["Sim_PL"],ascending=True)

#lookup in 3rd worset

VaR = df.iloc[2]["Sim_PL"]* -1

print("VaR of the bond is:", np.round(VaR,2))

VaR of the bond is: 1.91
